In [ ]:
import pandas as pd
from numpy.linalg import norm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit

In [ ]:
'''
np.corrcoef
Return Pearson product-moment correlation coefficients.
'''

In [ ]:
def Average(lst): 
	return round(sum(lst) / len(lst),4) 

In [ ]:
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
entity_user = 943
entity_item = 1682
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)


In [33]:
user_business = np.load('../hw3_data/Yelp/user_business.npy')
X = user_business[:,:2]
y = user_business[:,-1]
entity_user = 16239
entity_item = 14284
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)


In [38]:
user_book = np.load('../hw3_data/Douban_Book/user_book.npy')

X = user_book[:,:2]
y = user_book[:,-1]
entity_user = 13024
entity_item = 22347
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)

In [40]:
#User-based CF
RMSE_M = []
recall_M = []
NDCG_M = []
weight_sum = True
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    item_rating = np.zeros((entity_item, entity_user), dtype=int)
    for i in range(y_train.shape[0]):
        user, item = X_train[i]
        item_rating[item-1, user-1] = y_train[i]
        
    ICF = np.corrcoef(item_rating)
    #ICF = cosine_similarity(item_rating)
    ICF[np.isnan(ICF)] = 0

    Mu = item_rating.sum()/np.count_nonzero(item_rating)
    dev_user, dev_item = np.std(item_rating,axis=0), np.std(item_rating,axis=1)

    predict = []
    for i in range(X_test.shape[0]):

        user, item =  X_test[i]
        most_similar_item = np.argpartition(ICF[item-1],-6)[-6:-1]
        msi_weight = ICF[item-1][most_similar_item]
        rank = item_rating[most_similar_item, user-1]

        if weight_sum == True:
            bu = dev_user[user-1]-Mu
            bi = dev_item[item-1]-Mu
            bj = dev_item[most_similar_item]-Mu
            bui = Mu + bu + bi
            buj = Mu + bu + bj
            weight_rank = (rank-buj)*msi_weight
            # weight_rank = rank*msi_weight
            if msi_weight.sum() == 0 :
                predict.append(0)
            else:
                # pred = bui+ weight_rank.sum()/msi_weight.sum()
                pred = weight_rank.sum()/msi_weight.sum()
                predict.append(pred)
        else:
            pred = rank.sum()/5
            predict.append(pred)
    
    RMSE = mean_squared_error(np.array(predict),y_test, squared=False)
    RMSE_M.append(RMSE)
    recall = recall_score(np.round(np.array(predict)),y_test, average='micro')
    recall_M.append(recall)
    NDCG = ndcg_score(np.array([predict]),y_test.reshape(1, -1), k=10)
    NDCG_M.append(NDCG) 
    print(f"RMSE: {RMSE}, Recall: {recall}, NDCG: {NDCG}")
print(f"Average RMSE: {Average(RMSE_M)}, Average Recall: {Average(recall_M)}, Average NDCG: {Average(NDCG_M)}")    


RMSE: 1.1845996132128827, Recall: 0.3182136037223313, NDCG: 0.48038195458593536
RMSE: 1.1955258151307064, Recall: 0.3159140421182862, NDCG: 0.4801534308867931
RMSE: 1.1919513820662302, Recall: 0.31717812861407707, NDCG: 0.48285198893389125
RMSE: 1.1827773090517415, Recall: 0.31837497646647483, NDCG: 0.4818143259783191
RMSE: 1.188011021120685, Recall: 0.3178505150480084, NDCG: 0.4818504324651894
Average RMSE: 1.1886, Average Recall: 0.3175, Average NDCG: 0.4814


In [ ]:
# 扣除所有使用者的加總平均，undo
mean_by_user = user_rating.sum(axis=1)/np.count_nonzero(user_rating, axis=1)
mean_by_user
none_zero = np.where(user_rating > 0)
none_zero[]

# try